In [109]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [103]:
# Load data
train = pd.read_csv("/content/drive/MyDrive/Meidcal Chatbot/Training.csv")
test = pd.read_csv("/content/drive/MyDrive/Meidcal Chatbot/Testing.csv")

# Split data into input features and target variable
X = train.iloc[:, 0:132].values
y = train.iloc[:, -1].values

train

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


In [104]:
# Encode the target variable using LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [105]:
# Define the decision tree classifier
pipe = make_pipeline(StandardScaler(), DecisionTreeClassifier())

# Define the parameter grid for grid search
param_grid = {'decisiontreeclassifier__max_depth': range(1, 20),
              'decisiontreeclassifier__min_samples_split': range(2, 10),
              'decisiontreeclassifier__min_samples_leaf': range(1, 10),
              'decisiontreeclassifier__criterion': ['gini', 'entropy']}

# Perform grid search using cross-validation
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)

In [111]:
grid.fit(X_train, y_train)

print("Best parameters: ", grid.best_params_)
scores = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print("Cross-validation scores: ", scores)

def predict_disease(symptoms):
    symptoms_arr = np.zeros((1, len(dr.columns)))
    for symptom in symptoms:
        symptoms_arr[0, dr.columns.get_loc(symptom)] = 1
    
    disease_id = grid.best_estimator_.predict(symptoms_arr)[0]
    disease = labelencoder.inverse_transform([disease_id])[0]
    
    relevant_symptoms = list(dr.loc[disease][dr.loc[disease] == 1].index)
    
    return (disease, relevant_symptoms)

# Define a function to interact with the user
def chat():
    print("Hello, I'm Medi-Bot.")
    print('I can help you predict the disease based on the symptoms you have.')
    print("Please enter your symptoms one by one and enter 'done' when you're finished.")
    symptoms = []
    while True:
        symptom = input("Symptom: ")
        if symptom.lower() == "done":
            break
        if symptom in dr.columns:
            symptoms.append(symptom)
        else:
            print("I'm sorry, I don't recognize that symptom. Please try again.")
    if len(symptoms) == 0:
        print("I'm sorry, you haven't entered any symptoms.")
    else:
        disease, relevant_symptoms = predict_disease(symptoms)
        if len(relevant_symptoms) > 1:
            print("Based on the symptoms you have entered, you may have", disease)
            print("Other symptoms you may be experiencing include:", relevant_symptoms)
        else:
            print("Based on the symptoms you have entered, you may have", disease)

Best parameters:  {'decisiontreeclassifier__criterion': 'entropy', 'decisiontreeclassifier__max_depth': 19, 'decisiontreeclassifier__min_samples_leaf': 1, 'decisiontreeclassifier__min_samples_split': 4}
Cross-validation scores:  [0.92276423 0.97154472 0.93089431 0.91463415 0.93495935]


In [113]:
chat()

Hello, I'm Medi-Bot.
I can help you predict the disease based on the symptoms you have.
Please enter your symptoms one by one and enter 'done' when you're finished.
Symptom: shivering
Symptom: chills
Symptom: acidity
Symptom: done
Based on the symptoms you have entered, you may have Allergy
Other symptoms you may be experiencing include: ['continuous_sneezing', 'shivering', 'chills', 'watering_from_eyes']
